In [1]:
import cv2 as cv
import mediapipe as mp 

2024-06-28 23:17:46.659720: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-28 23:17:46.691038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 23:17:46.867092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 23:17:46.868577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 23:17:47.622612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands #this is the actual model 


#Also in CV y axis is increasing downwards and decreasing upward
#We are here defining a function to tell what are the hand position invovled.


def getHandMove(hand_landmarks):
    landmarks= hand_landmarks.landmark
    if all([landmarks[i].y < landmarks[i+3].y for i in range (9,20,4)]):return "rock"   #this is from the mediapipe figure
    elif landmarks[13].y < landmarks[16].y and landmarks[17].y<landmarks[20].y:return "scissors"
    else:return "paper"
    


Now we are setting up the camera


In [3]:
clock=0
p1_move=p2_move=None
gameText=""
success=True 

In [4]:
vid = cv.VideoCapture(0)


cv.namedWindow('frame', cv.WINDOW_NORMAL)

# Resize the window to the desired size (width, height)
cv.resizeWindow('frame', 800, 600)
font_scale = 0.5

with mp_hands.Hands(model_complexity=0,min_detection_confidence = 0.5,min_tracking_confidence=0.5) as hands:
    while True:
        ret,frame = vid.read()
        if not ret or frame is None:break  ## now we change into rgb colour as it is BGR

        frame= cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        results=hands.process(frame) #This runs model and find where our hand are

        frame=cv.cvtColor(frame,cv.COLOR_RGB2BGR) #revoting to same colour

        if results.multi_hand_landmarks:      #superimposing landmarks into the hands
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                          mp_drawing_styles.get_default_hand_landmarks_style(),
                                          mp_drawing_styles.get_default_hand_connections_style())
        frame=cv.flip(frame,1) #this is to flip the camera 


        if 0<= clock <20:
            success=True
            gameText="Ready?"
        elif clock <30: gameText="3..."
        elif clock <40: gameText="2..."
        elif clock <50: gameText="1..."    
        elif clock <60: gameText="GO!!"
        elif clock == 60:
            hls= results.multi_hand_landmarks    #here we do hand detection
            if hls and len(hls)==2:              #checking if hls is actual or null object and if not empty is there 2 hands
                p1_move= getHandMove(hls[0])
                p2_move = getHandMove(hls[1])
            else:
                success=False

        
        elif clock <100:
            if success:
                gameText=f"Played 1 played {p1_move}. Player 2 played {p2_move}."
                if p1_move == p2_move:gameText =f"{gameText} Game is tied"
                elif p1_move =='paper' and p2_move == 'rock':gameText=f'{gameText} Player 1 wins.'
                elif p1_move =='rock' and p2_move == 'scissors':gameText=f'{gameText} Player 1 wins.'
                elif p1_move =='scissors' and p2_move == 'paper':gameText=f'{gameText} Player 1 wins.'
                else: gameText=f'{gameText} Player 2 wins.'
            else:
                gameText="Didn't play properly!"
        
        cv.putText(frame,f'Clock:{clock}',(50,50),cv.FONT_HERSHEY_PLAIN,1,(0,255,255),1,cv.LINE_AA)
        cv.putText(frame,gameText,(50,80),cv.FONT_HERSHEY_PLAIN,1,(0,255,255),1,cv.LINE_AA)
        clock=(clock+1) % 100
        cv.imshow('frame',frame)
        
        if cv.waitKey(1) & 0xFF==ord('q'):break  ##This is to close the camera type q
vid.release()
cv.destroyAllWindows()






I0000 00:00:1719595969.966934   40707 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719595969.969498   40766 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719595969.998945   40755 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719595970.005895   40758 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/aasharya/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is